In [ ]:
import boto3
import numpy as np
import os
import pathlib
import random
import tensorflow as tf
import time

from keras.preprocessing.image import img_to_array, load_img
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.tensorflow import TensorFlow
from sklearn.model_selection import train_test_split
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner

from IPython.display import Image

# PHASE 1 : BUILD

# Converting images to the TFRecord format

## Get all image paths and shuffle them

In [ ]:
data_root = pathlib.Path(os.path.join(os.getcwd(), 'images'))

In [ ]:
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)
image_count = len(all_image_paths)

In [ ]:
all_image_paths[:10]

## Get all image labels

In [ ]:
all_image_labels = [int(pathlib.Path(path).parent.name) for path in all_image_paths]

In [ ]:
all_image_labels[:10]

## Get a numpy array containing all images with their associated labels

In [ ]:
channels = 3
image_height = 50
image_width = 50

In [ ]:
dataset = np.ndarray(shape=(image_count, image_height, image_width, channels),
                     dtype=np.uint8)

i = 0
for file in all_image_paths:
    img = load_img(file)  # this is a PIL image
    img = img.resize((image_width, image_height))
    x = img_to_array(img, 'channels_last') 
    dataset[i] = x
    i += 1
    if i % 250 == 0:
        print("%d images to array" % i)
print("All images to array!")

## Split dataset into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, all_image_labels, test_size=0.3, random_state=33)

## Convert images

In [ ]:
def convert_to_tfrecord(images, labels, num_examples, name, directory):
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

    if images.shape[0] != num_examples:
        raise ValueError('Images size %d does not match label size %d.' % (images.shape[0], num_examples))
    rows = images.shape[1]
    cols = images.shape[2]
    depth = images.shape[3]

    filename = os.path.join(directory, name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tobytes()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(labels[index]),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
convert_to_tfrecord(X_train, y_train, len(y_train), 'images_train', os.getcwd())

In [ ]:
convert_to_tfrecord(X_test, y_test, len(y_test), 'images_test', os.getcwd())

## Upload the train and test .tfrecords files to S3

In [ ]:
# %%bash
bucket = 'sagemaker-projects-demo'

!aws s3 cp images_train.tfrecords s3://{bucket}/breast-cancer-detection/input/tfrecord/train/
!aws s3 cp images_test.tfrecords s3://{bucket}/breast-cancer-detection/input/tfrecord/test/

In [ ]:
Image(filename='img/1_TF_Process.png')

# Creating a model using Tensorflow

## Configure hyperparameters

In [ ]:
# Number of output classes
num_classes = 2

# Batch size for training
mini_batch_size =  128

# Epochs for training
epochs = 2

# Learning rate
learning_rate = 0.01

## Create a unique job name 

In [ ]:
job_name_prefix = 'breast-cancer-detection'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Specify the input paths for the job

In [ ]:
bucket = 'sagemaker-projects-demo'
input_prefix = 'breast-cancer-detection/input/tfrecord'
input_train = 's3://{}/{}/train/'.format(bucket, input_prefix)
input_test = 's3://{}/{}/test/'.format(bucket, input_prefix)

## Specify the output path for the job

In [ ]:
output_prefix = 'breast-cancer-detection/output'
output_path = 's3://{}/{}/'.format(bucket, output_prefix)

## Configure training instances

In [ ]:
instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size_gb = 50

## Get the execution role

In [ ]:
role = get_execution_role()

## Configure train timeout

In [ ]:
train_timeout = 360000

## Specify the path to the training script

In [ ]:
training_script_path = 'tensorflow2Script.py'

## Create a sagemaker.TensorFlow estimator

In [ ]:
estimator = TensorFlow(entry_point=training_script_path,
                       role=role,
                       train_instance_count=instance_count,
                       train_instance_type=instance_type,
                       train_volume_size=volume_size_gb,
                       train_max_run=train_timeout,
                       model_dir=output_path,
                       output_path=output_path,
                       framework_version='2.0.0',
                       py_version = 'py3',
                       hyperparameters = {
                           'num-classes': num_classes,
                           'mini-batch-size': mini_batch_size,
                           'epochs': epochs,
                           'learning-rate': learning_rate,
                           'train-size': len(y_train),
                           'test-size': len(y_test)
                       },
                       metric_definitions = [
                           {
                               'Name': 'loss',
                               'Regex': 'loss: ([0-9\\.]+)'
                           }
                       ])

# PHASE 2 : TRAINING


## Part - 1

## Create a training job

In [ ]:
estimator.fit({
    'train': input_train,
    'test': input_test
}, job_name = job_name)

## Part - 2 (with HPO)

## Defining tuning configuration

In [ ]:
hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.001, 1.0)
}
objective_metric_name = 'loss'
objective_type = 'Minimize'

max_jobs=2
max_parallel_jobs=2

## Create a unique job name

In [ ]:
job_name_prefix = 'tuning-tf'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Creating a hyperparameter tuner

In [ ]:
tuner = HyperparameterTuner(estimator=estimator, 
                            objective_metric_name=objective_metric_name, 
                            hyperparameter_ranges=hyperparameter_ranges,
                            objective_type=objective_type, 
                            max_jobs=max_jobs, 
                            max_parallel_jobs=max_parallel_jobs,
                            metric_definitions = [
                                {
                                   'Name': 'loss',
                                   'Regex': 'loss: ([0-9\\.]+)'
                                }
                            ])

## Launch the tuning job

In [ ]:
tuner.fit({
    'train': input_train,
    'test': input_test
}, job_name = job_name)
tuner.wait()